<a href="https://colab.research.google.com/github/sthanikan2000/CS3121-Project/blob/main/DS_Proj_leaves_preprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [206]:
import pandas as pd

In [207]:
holi=pd.read_csv('holidays.csv')
df=pd.read_csv('leaves.csv')

In [208]:
holi= pd.concat([pd.Series([holi.columns[0]]), holi[holi.columns[0]]], ignore_index=True)


In [209]:
holi

0      2017-05-07
1      2017-05-14
2      2017-05-21
3      2017-05-28
4      2017-06-04
          ...    
116    2022-12-25
117    2022-12-29
118    2022-12-30
119    2022-07-13
120    2023-01-06
Length: 121, dtype: object

In [210]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1018 entries, 0 to 1017
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Employee_No   1018 non-null   int64 
 1   leave_date    1018 non-null   object
 2   Type          1018 non-null   object
 3   Applied Date  1018 non-null   object
 4   Remarks       773 non-null    object
 5   apply_type    1018 non-null   object
dtypes: int64(1), object(5)
memory usage: 47.8+ KB


In [211]:
df.nunique()

Employee_No      70
leave_date      367
Type              2
Applied Date    220
Remarks         135
apply_type        2
dtype: int64

In [212]:
(df["apply_type"]=="Anual").sum()

466

In [213]:
(df["Type"]=="Half Day").sum()

376

In [214]:
df['leave_date'] = pd.to_datetime(df['leave_date'])
df['Applied Date'] = pd.to_datetime(df['Applied Date'])

mask= df["Applied Date"] > df["leave_date"]
df[mask].shape
# The above returned 974 records out of 1018
# Hence most of the leaves are not informed before taking a leave

(974, 6)

In [215]:
df.shape

(1018, 6)

In [216]:
mask=df['leave_date'].isin(holi)

In [217]:
df=df[~mask].copy() # Remove holidays from leaves
df

,Employee_No,leave_date,Type,Applied Date,Remarks,apply_type
0,347,2020-01-29,Half Day,2022-01-04 15:33:00,\N,Anual
1,347,2020-01-29,Half Day,2022-01-04 15:33:00,\N,Anual
2,347,2020-01-29,Half Day,2022-01-04 15:36:00,\N,Anual
3,347,2020-01-29,Half Day,2022-01-04 15:36:00,\N,Anual
4,347,2020-01-29,Half Day,2022-01-04 18:02:00,\N,Anual
...,...,...,...,...,...,...
1013,596,2023-01-27,Full Day,2023-01-31 00:00:00,Personal,Casual
1014,596,2023-01-28,Half Day,2023-01-31 00:00:00,Personal,Casual
1015,2972,2023-01-28,Half Day,2023-02-03 14:00:00,Personal,Casual
1016,2972,2023-01-30,Full Day,2023-02-03 14:00:00,Personal,Casual


In [218]:
df.drop(["Remarks",'Applied Date'], axis=1, inplace=True)

df.drop_duplicates(inplace=True)
df.shape

(959, 4)

In [219]:
# Identify duplicate records based on "Employee_No," "leave_date," and "Type"
duplicates = df.duplicated(subset=['Employee_No', 'leave_date', 'Type'], keep=False)

# Update "apply_type" to null for duplicate records
df.loc[duplicates, 'apply_type'] = None

# Calculate the mode of "apply_type" for each unique "Employee_No"
mode_per_employee = df.groupby('Employee_No')['apply_type'].apply(lambda x: x.mode()[0])

# Fill missing values in "apply_type" with the mode of each employee
df['apply_type'] = df.apply(lambda row: mode_per_employee[row['Employee_No']] if pd.isnull(row['apply_type']) else row['apply_type'], axis=1)

df.drop_duplicates(inplace=True)


In [220]:
df

,Employee_No,leave_date,Type,apply_type
0,347,2020-01-29,Half Day,Anual
5,1961,2022-01-24,Full Day,Anual
6,1961,2022-01-25,Full Day,Anual
7,2346,2022-01-10,Full Day,Casual
8,2346,2022-01-11,Full Day,Casual
...,...,...,...,...
1013,596,2023-01-27,Full Day,Casual
1014,596,2023-01-28,Half Day,Casual
1015,2972,2023-01-28,Half Day,Casual
1016,2972,2023-01-30,Full Day,Casual


In [221]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 947 entries, 0 to 1017
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Employee_No  947 non-null    int64         
 1   leave_date   947 non-null    datetime64[ns]
 2   Type         947 non-null    object        
 3   apply_type   947 non-null    object        
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 37.0+ KB


In [222]:
# Identify duplicate records based on "Employee_No," "leave_date," and "apply_type"
duplicates = df.duplicated(subset=['Employee_No', 'leave_date', 'apply_type'], keep=False)

# Update "apply_type" to null for duplicate records
df.loc[duplicates, 'Type'] = None

In [223]:
df.info()
# Type attribute doesn't have any null.
#So, there are no situations like a employee take a half day and full day in same date.

<class 'pandas.core.frame.DataFrame'>
Int64Index: 947 entries, 0 to 1017
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Employee_No  947 non-null    int64         
 1   leave_date   947 non-null    datetime64[ns]
 2   Type         947 non-null    object        
 3   apply_type   947 non-null    object        
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 37.0+ KB


In [224]:
df

,Employee_No,leave_date,Type,apply_type
0,347,2020-01-29,Half Day,Anual
5,1961,2022-01-24,Full Day,Anual
6,1961,2022-01-25,Full Day,Anual
7,2346,2022-01-10,Full Day,Casual
8,2346,2022-01-11,Full Day,Casual
...,...,...,...,...
1013,596,2023-01-27,Full Day,Casual
1014,596,2023-01-28,Half Day,Casual
1015,2972,2023-01-28,Half Day,Casual
1016,2972,2023-01-30,Full Day,Casual


In [225]:
# Perform one-hot encoding on 'Type' and 'apply_type' columns
encoded_cols = pd.get_dummies(df[['Type', 'apply_type']])

# Add the encoded columns to the original dataframe
df = pd.concat([df, encoded_cols], axis=1)

# Remove the original 'Type' and 'apply_type' columns
df = df.drop(['Type', 'apply_type'], axis=1)

In [226]:
df

,Employee_No,leave_date,Type_Full Day,Type_Half Day,apply_type_Anual,apply_type_Casual
0,347,2020-01-29,0,1,1,0
5,1961,2022-01-24,1,0,1,0
6,1961,2022-01-25,1,0,1,0
7,2346,2022-01-10,1,0,0,1
8,2346,2022-01-11,1,0,0,1
...,...,...,...,...,...,...
1013,596,2023-01-27,1,0,0,1
1014,596,2023-01-28,0,1,0,1
1015,2972,2023-01-28,0,1,0,1
1016,2972,2023-01-30,1,0,0,1


In [227]:
# Group the dataframe by 'Employee_No' and calculate the sum of desired columns
grouped_df = df.groupby('Employee_No').agg({
    'Type_Full Day': 'sum',
    'Type_Half Day': 'sum',
    'apply_type_Anual': 'sum',
    'apply_type_Casual': 'sum'
}).reset_index()


In [228]:
grouped_df

,Employee_No,Type_Full Day,Type_Half Day,apply_type_Anual,apply_type_Casual
0,347,16,6,11,11
1,348,5,7,5,7
2,351,4,6,0,10
3,356,6,6,0,12
4,373,15,6,14,7
...,...,...,...,...,...
64,2737,3,1,2,2
65,2813,1,0,0,1
66,2831,0,1,0,1
67,2972,1,2,0,3


In [229]:
grouped_df.to_csv('leaves_preprocessed_200635E.csv', index=False)